# LOAD Modules

In [54]:
%matplotlib inline
#%matplotlib widget
%load_ext autoreload
%autoreload 2


import numpy as np
from visualisations import eeg_sample_plot, events_distribution_plot
from sklearn.model_selection import train_test_split
from torchvision.transforms import Compose, ToTensor
from torch.utils.data import DataLoader
from datasets import ChannelsVoltageDataset
from neural_nets.nn_models_getter import get_nn_model
from optimizers import get_optimizer
from utils_train import fit, test, plot_all_metrics, plot_val_metrics
from configs import configs_ozhan, configs_tim, configs_joaquin
from data_loader_creation import get_dataloader_objects
from classification_results import results_storer



The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


# Select the configuration

In [55]:
""" USER: SELECT THE CONFIGURATION YOU NEED """
#myList = configs_tim.list_of_configs
myList = configs_joaquin.list_of_configs
#myList = configs_ozhan.list_of_configs

# Train

In [62]:
training_curves = {}

for my_cfg in myList:
    
    curve_name = my_cfg.nn_list[my_cfg.nn_selection_idx] + "_" + my_cfg.optimizer_list[my_cfg.optimizer_selection_idx]
    print(curve_name,"\n-------------------------\n", flush=True)
    
    """ PREPARE DATALOADERS """
    train_dl, val_dl, test_dl, input_dimension_, output_dimension_ = get_dataloader_objects(my_cfg)


    """CLASSIFICATION"""
    # Get the model
    model_untrained = get_nn_model(my_cfg.nn_list[my_cfg.nn_selection_idx], input_dimension=input_dimension_,
                                   output_dimension=output_dimension_, dropout=my_cfg.dropout_perc)
    
    # Get the optimizer
    optimizer = get_optimizer(my_cfg.optimizer_list[my_cfg.optimizer_selection_idx], my_cfg.learning_rate,
                              model_untrained.parameters(), my_cfg.momentum, my_cfg.weight_decay)

    # Train and show validation loss
    train_losses, train_accuracies, val_losses, val_accuracies, model_trained, time_spent_for_training_s =\
        fit(train_dl, val_dl, model_untrained, optimizer, my_cfg.loss_fn, my_cfg.num_of_epochs, \
            scheduler=my_cfg.scheduler, apply_early_stopping = my_cfg.use_early_stopping, \
            estop_patience=my_cfg.es_patience, curve_name = curve_name)
    
    
    curves = train_losses, train_accuracies, val_losses, val_accuracies, model_trained, time_spent_for_training_s
    
    
    training_curves[curve_name] = curves
    
    # Test the net
    print('\nPerformance on the test set:')
    test_loss, test_accuracy = test(model_trained, test_dl, my_cfg.loss_fn, print_loss=True)
    print('->test_loss: {:.4f}, test_accuracy: {:.4f}%'.format(test_loss, test_accuracy))


    # Store the results
    results_storer.store_results_for_plot(my_cfg,test_loss, test_accuracy, train_losses,
                                 train_accuracies, time_spent_for_training_s, val_losses, val_accuracies)

    results_storer.store_results(my_cfg, model_trained, optimizer, test_loss, test_accuracy, train_losses,
                                 train_accuracies, time_spent_for_training_s, val_losses, val_accuracies, test_dl)


plot_all_metrics(training_curves)

ConvNet3DFFT_SGD 
-------------------------

We are not on the cluster...

Data is being loaded using MNE...



100%|██████████| 6/6 [00:13<00:00,  2.53s/it]


2528 matching events found
No baseline correction applied
...data loading with MNE was finished. 

cuda will be used for training of this model.
Learning Rate:  0.001 

torch.Size([128, 64, 817])
torch.Size([128, 1, 52288])


RuntimeError: shape '[128, 1, 1]' is invalid for input of size 6692864